<a href="https://colab.research.google.com/github/naveenkumar1805/capstone-project-2-supervised-machine-learning-_retail_Sales_prediction/blob/main/Rossmann_Sales_Prediction_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <b><u> Project Title : Sales Prediction : Predicting sales of a major store chain Rossmann</u></b>

## <b> Problem Description </b>

### Rossmann operates over 3,000 drug stores in 7 European countries. Currently, Rossmann store managers are tasked with predicting their daily sales for up to six weeks in advance. Store sales are influenced by many factors, including promotions, competition, school and state holidays, seasonality, and locality. With thousands of individual managers predicting sales based on their unique circumstances, the accuracy of results can be quite varied.

### You are provided with historical sales data for 1,115 Rossmann stores. The task is to forecast the "Sales" column for the test set. Note that some stores in the dataset were temporarily closed for refurbishment.


## <b> Data Description </b>

### <b>Rossmann Stores Data.csv </b> - historical data including Sales
### <b>store.csv </b> - supplemental information about the stores


### <b><u>Data fields</u></b>
### Most of the fields are self-explanatory. The following are descriptions for those that aren't.

* #### Id - an Id that represents a (Store, Date) duple within the test set
* #### Store - a unique Id for each store
* #### Sales - the turnover for any given day (this is what you are predicting)
* #### Customers - the number of customers on a given day
* #### Open - an indicator for whether the store was open: 0 = closed, 1 = open
* #### StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None
* #### SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools
* #### StoreType - differentiates between 4 different store models: a, b, c, d
* #### Assortment - describes an assortment level: a = basic, b = extra, c = extended
* #### CompetitionDistance - distance in meters to the nearest competitor store
* #### CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened
* #### Promo - indicates whether a store is running a promo on that day
* #### Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store is participating
* #### Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2
* #### PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pylab
from scipy import stats

In [5]:
rossmann_store_data=pd.read_csv('/content/drive/MyDrive/Copy of Rossmann Stores Data.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [16]:
store_df= pd.read_csv("/content/drive/MyDrive/capstone_project 2-supervised_ml/Copy of store.csv")

In [7]:
rossmann_store_data.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [8]:
rossmann_store_data.isnull().sum()

Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

In [17]:
store_df.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [18]:
store_df.isnull().sum()

Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            3
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64

In [10]:
store_df= store_df.fillna(0)
store_df

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,0.0,0.0,0
4,5,a,a,29910.0,4.0,2015.0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...
1110,1111,a,a,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct"
1111,1112,c,c,1880.0,4.0,2006.0,0,0.0,0.0,0
1112,1113,a,c,9260.0,0.0,0.0,0,0.0,0.0,0
1113,1114,a,c,870.0,0.0,0.0,0,0.0,0.0,0


In [19]:
[features for features in store_df.columns if store_df[features].isnull().sum()>0]

['CompetitionDistance',
 'CompetitionOpenSinceMonth',
 'CompetitionOpenSinceYear',
 'Promo2SinceWeek',
 'Promo2SinceYear',
 'PromoInterval']

In [33]:
store_df['Store'].nunique

<bound method IndexOpsMixin.nunique of 0          1
1          2
2          3
3          4
4          5
        ... 
1110    1111
1111    1112
1112    1113
1113    1114
1114    1115
Name: Store, Length: 1115, dtype: int64>

In [31]:
for col in store_df.columns:
  print(store_df[col].unique)

<bound method Series.unique of 0          1
1          2
2          3
3          4
4          5
        ... 
1110    1111
1111    1112
1112    1113
1113    1114
1114    1115
Name: Store, Length: 1115, dtype: int64>
<bound method Series.unique of 0       c
1       a
2       a
3       c
4       a
       ..
1110    a
1111    c
1112    a
1113    a
1114    d
Name: StoreType, Length: 1115, dtype: object>
<bound method Series.unique of 0       a
1       a
2       a
3       c
4       a
       ..
1110    a
1111    c
1112    c
1113    c
1114    c
Name: Assortment, Length: 1115, dtype: object>
<bound method Series.unique of 0        1270.0
1         570.0
2       14130.0
3         620.0
4       29910.0
         ...   
1110     1900.0
1111     1880.0
1112     9260.0
1113      870.0
1114     5350.0
Name: CompetitionDistance, Length: 1115, dtype: float64>
<bound method Series.unique of 0        9.0
1       11.0
2       12.0
3        9.0
4        4.0
        ... 
1110     6.0
1111     4.0
1112     Na

In [37]:
store_df.describe(include='all')

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
count,1115.00000,1115,1115,1112.000000,761.000000,761.000000,1115.000000,571.000000,571.000000,571
unique,NaN,4,3,NaN,NaN,NaN,NaN,NaN,NaN,3
top,NaN,a,a,NaN,NaN,NaN,NaN,NaN,NaN,"Jan,Apr,Jul,Oct"
freq,NaN,602,593,NaN,NaN,NaN,NaN,NaN,NaN,335
mean,558.00000,NaN,NaN,5404.901079,7.224704,2008.668857,0.512108,23.595447,2011.763573,NaN
std,322.01708,NaN,NaN,7663.174720,3.212348,6.195983,0.500078,14.141984,1.674935,NaN
min,1.00000,NaN,NaN,20.000000,1.000000,1900.000000,0.000000,1.000000,2009.000000,NaN
25%,279.50000,NaN,NaN,717.500000,4.000000,2006.000000,0.000000,13.000000,2011.000000,NaN
50%,558.00000,NaN,NaN,2325.000000,8.000000,2010.000000,1.000000,22.000000,2012.000000,NaN
75%,836.50000,NaN,NaN,6882.500000,10.000000,2013.000000,1.000000,37.000000,2013.000000,NaN


In [ ]:
df=pd.merge(rossmann_store_data,store_df,how='inner', on='Store')

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.tail()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'] )
df['Promo2SinceYear']= pd.to_datetime(df['Promo2SinceYear'])

In [ ]:
df['year_of_sales'] =  df['Date'].dt.year 
df['month_of_sales'] =  df['Date'].dt.month.astype(int)
df['day_of_sales'] =  df['Date'].dt.day.astype(int)
df['Promo2SinceYear'] =  df['Promo2SinceYear'].dt.year.astype(int)

df.head(5)

In [ ]:
df = df.drop(['Date'],axis = 1)

In [ ]:
df.info()

In [ ]:
df = pd.get_dummies(df,drop_first=True)
df.head()

In [ ]:
#withought using min_max scaling
X = df.drop('Sales',axis = 1)
y = df['Sales']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#Importing neccesary packages
from sklearn.linear_model import LinearRegression
from sklearn import metrics
print(X_train.shape)
print(X_test.shape)

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
#Checking accuracy on train data
accuracy = model.score(X_train,y_train)
print(accuracy*100,'%')

In [ ]:
#Checking accuracy on test data
accuracy = model.score(X_test,y_test)
print(accuracy*100,'%')

In [ ]:
y_pred = model.predict(X_test)

metrics.r2_score(y_pred,y_test)

In [ ]:
plt.figure(figsize=(20,5))
plt.plot(y_pred)
plt.plot(np.array(y_test))
plt.legend(["Predicted","Actual"])
plt.show()

In [ ]:
#Normalization/scaling of data - understanding scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)
scaled_data

In [ ]:
#converting data back to pandas dataframe
df_scaled = pd.DataFrame(scaled_data)
df_scaled.columns = df.columns

In [ ]:
X = df_scaled.drop('Sales',axis = 1)
y = df_scaled['Sales']

In [ ]:
#Dividing data in test and train
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#Importing neccesary packages
from sklearn.linear_model import LinearRegression
from sklearn import metrics
print(X_train.shape)
print(X_test.shape)

In [ ]:
#Fitting lineaar regression model
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
#Checking accuracy on train data
accuracy = model.score(X_train,y_train)
print(accuracy*100,'%')

In [ ]:
#Checking accuracy on test data
accuracy = model.score(X_test,y_test)
print(accuracy*100,'%')

In [ ]:
y_pred = model.predict(X_test)

metrics.r2_score(y_pred,y_test)

In [ ]:
# Fitting Random Forest Regression to the dataset
# import the regressor
from sklearn.ensemble import RandomForestRegressor
 
 # create regressor object
regressor = RandomForestRegressor(n_estimators = 10, random_state = 0)
 
# fit the regressor with x and y data
regressor.fit(X, y) 

In [ ]:
#Checking accuracy on train data
accuracy1 = regressor.score(X_train,y_train)
print(accuracy*100,'%')

In [ ]:
#Checking accuracy on test data
accuracy2 = model.score(X_test,y_test)
print(accuracy*100,'%')

In [ ]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
df['CompetitionOpenSinceYear'].value_counts().reset_index().rename(columns={'index':'year'})

In [ ]:
df.shape

In [ ]:
len(df[df.duplicated()])

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
# Dependent variable 'Sales'
plt.figure(figsize=(7,7))
sns.distplot(df['Sales'],color="g")

In [ ]:
columns_to_fix = ['Sales','Customers','CompetitionDistance']

In [ ]:
#Defining percentile range and calulating IQR
Q1 = df[columns_to_fix].quantile(0.25)    #25% percentile
Q3 = df[columns_to_fix].quantile(0.75)    #75% percentile
IQR = Q3 - Q1     #IQR (Inter Quartile Range)
print(IQR)

In [ ]:
df = df[~((df[columns_to_fix] < (Q1 - 1.5 * IQR)) |(df[columns_to_fix] > (Q3 + 1.5 * IQR))).any(axis=1)]

In [ ]:
df.describe()

In [ ]:
import math

In [ ]:
plt.figure(figsize=(7,7))
sns.distplot(np.sqrt([df['Sales']]),color="g")

In [ ]:
df.describe()

In [ ]:
df = df.loc[(df != 0).any(axis=1)]

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
plt.figure(figsize=(7,7))
sns.distplot(np.sqrt([df['Sales']]),color="g")